In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as si
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations, initializers
from tensorflow.keras.layers import BatchNormalization

np.set_printoptions(precision = 3, suppress = True)
import warnings
warnings.filterwarnings("ignore")

In [2]:
ES_EZ = pd.read_csv('/Users/gaojinglun/Desktop/RSG/ES_EZ.csv')
ES_EZ = ES_EZ.drop(['Unnamed: 0'], axis = 1)
df = ES_EZ[['Put.Call', 'Strike.Price', 'Settlement', 'Historical Vol', 
            'futures.price', 'Time.to.maturity', 'Risk.Free.Rate', 'Implied.Volatility']]
df = df.dropna()
df.head()

,Put.Call,Strike.Price,Settlement,Historical Vol,futures.price,Time.to.maturity,Risk.Free.Rate,Implied.Volatility
0,C,2200,1059.0,0.008335,3259.0,0.213699,0.0154,0.338267
1,C,2250,1009.1,0.008335,3259.0,0.213699,0.0154,0.326829
2,C,2270,989.1,0.008335,3259.0,0.213699,0.0154,0.323456
3,C,2300,959.2,0.008335,3259.0,0.213699,0.0154,0.317834
4,C,2310,949.2,0.008335,3259.0,0.213699,0.0154,0.314282


In [3]:
ES_EZ_call = df[df['Put.Call'] == 'C']
Call_temp = 100 * ES_EZ_call['Strike.Price'].values / ES_EZ_call['futures.price'].values
Call_ITM_idx = Call_temp < 95.2
Call_ATM_idx = np.logical_and(Call_temp < 103, Call_temp >= 95.2)
Call_OTM_idx = Call_temp > 103
print('The number of ITM Call options is', np.sum(Call_ITM_idx))
print('The number of ATM Call options is', np.sum(Call_ATM_idx))
print('The number of OTM Call options is', np.sum(Call_OTM_idx))

The number of ITM Call options is 6678
The number of ATM Call options is 2223
The number of OTM Call options is 4807


In [4]:
# Add the moneyness indicator 
Call_moneyness = np.array(['ITM'] * ES_EZ_call.shape[0])
Call_moneyness[Call_ATM_idx] = 'ATM'
Call_moneyness[Call_OTM_idx] = 'OTM'

ES_EZ_call_X = ES_EZ_call[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Historical Vol']]
ES_EZ_call_y = ES_EZ_call['Implied.Volatility']
ES_EZ_call.shape

(13708, 8)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_call_X, ES_EZ_call_y, 
                                                    test_size = 0.2, stratify = Call_moneyness, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10966, 5), (2742, 5), (10966,), (2742,))

In [6]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))
X_train.head()

,futures.price,Strike.Price,Time.to.maturity,Risk.Free.Rate,Historical Vol
0,0.417376,0.485714,0.042980,0.063291,0.120078
1,0.441082,0.567347,0.478510,0.094937,0.189638
2,0.758969,0.522449,0.083095,0.044304,0.079680
3,0.428977,0.424490,0.002865,0.082278,0.244963
4,0.633945,0.634694,0.000000,0.044304,0.087062


In [7]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs = {}):
        self.times = []

    def on_epoch_begin(self, batch, logs = {}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs = {}):
        self.times.append(time.time() - self.epoch_time_start)

In [8]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
score = []
for i in range(10):
    learning_rate = [1e-2, 1e-3]
    score_small = []
    for lr in learning_rate:
        model = models.Sequential()
        # Input layer
        model.add(layers.Dense(units = 5,
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))

        numLayers = 3
        for i in range(numLayers):
            model.add(BatchNormalization())
            model.add(layers.Dense(units = 100, 
                                   kernel_initializer = initializers.RandomNormal(seed = 123),
                                   bias_initializer = initializers.Zeros()))
            model.add(layers.Activation(activations.elu))
            model.add(layers.Dropout(0.25))

        model.add(BatchNormalization())
        model.add(layers.Dense(units = 25, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

        # Output layer
        model.add(layers.Dense(units = 1, activation = "exponential"))

        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
        )

        time_callback = TimeHistory()

        history = model.fit(X_train, 
            y_train, 
            epochs = 100, 
            verbose = 0,
            batch_size = 32, 
            callbacks = [earlyStop, time_callback],
            validation_split = 0.2)
        BSNN_train_pred = model.predict(X_train)
        BSNN_test_pred = model.predict(X_test)
        trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
        testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
        score_small.append((trainr2, testr2))
    score.append(score_small)
score

[[(0.8785, 0.876), (0.8719, 0.8755)],
 [(0.839, 0.841), (0.8964, 0.8992)],
 [(0.8962, 0.9003), (0.837, 0.8431)],
 [(0.8786, 0.8832), (0.9356, 0.9361)],
 [(0.8915, 0.8968), (0.8898, 0.8919)],
 [(0.866, 0.8728), (0.9072, 0.9063)],
 [(0.9333, 0.9347), (0.912, 0.913)],
 [(0.7985, 0.8047), (0.909, 0.911)],
 [(0.8873, 0.8922), (0.9135, 0.9118)],
 [(0.8352, 0.8413), (0.9345, 0.9379)]]

In [9]:
learning_rate = [1e-2, 1e-3]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 5,
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    
    numLayers = 3
    for i in range(numLayers):
        model.add(BatchNormalization())
        model.add(layers.Dense(units = 100, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(layers.Dense(units = 25, 
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
    print("The training R squared is", 
          trainr2)
    testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
    print("The testing R squared is", 
          testr2)
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.7729
The testing R squared is 0.7772
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.944
The testing R squared is 0.9456
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_withLR0.001/assets


In [10]:
ES_EZ_put = df[df['Put.Call'] == 'P']
Put_temp = 100 * ES_EZ_put['Strike.Price'].values / ES_EZ_put['futures.price'].values
Put_ITM_idx = Put_temp > 103
Put_ATM_idx = np.logical_and(Put_temp < 103, Put_temp >= 95.2)
Put_OTM_idx = Put_temp < 95.2
print('The number of ITM Call options is', np.sum(Put_ITM_idx))
print('The number of ATM Call options is', np.sum(Put_ATM_idx))
print('The number of OTM Call options is', np.sum(Put_OTM_idx))

The number of ITM Call options is 3123
The number of ATM Call options is 2234
The number of OTM Call options is 10906


In [11]:
# Add the moneyness indicator 
Put_moneyness = np.array(['ITM'] * ES_EZ_put.shape[0])
Put_moneyness[Put_ATM_idx] = 'ATM'
Put_moneyness[Put_OTM_idx] = 'OTM'

ES_EZ_put_X = ES_EZ_put[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Historical Vol']]
ES_EZ_put_y = ES_EZ_put['Implied.Volatility']
ES_EZ_put.shape

(16263, 8)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_put_X, ES_EZ_put_y, 
                                                    test_size = 0.2, stratify = Put_moneyness, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13010, 5), (3253, 5), (13010,), (3253,))

In [13]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))

In [14]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
score = []
for i in range(10):
    learning_rate = [1e-2, 1e-3]
    score_small = []
    for lr in learning_rate:
        model = models.Sequential()
        # Input layer
        model.add(layers.Dense(units = 5,
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))

        numLayers = 3
        for i in range(numLayers):
            model.add(BatchNormalization())
            model.add(layers.Dense(units = 100, 
                                   kernel_initializer = initializers.RandomNormal(seed = 123),
                                   bias_initializer = initializers.Zeros()))
            model.add(layers.Activation(activations.elu))
            model.add(layers.Dropout(0.25))

        model.add(BatchNormalization())
        model.add(layers.Dense(units = 25, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

        # Output layer
        model.add(layers.Dense(units = 1, activation = "exponential"))

        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
        )

        time_callback = TimeHistory()

        history = model.fit(X_train, 
            y_train, 
            epochs = 100, 
            verbose = 0,
            batch_size = 32, 
            callbacks = [earlyStop, time_callback],
            validation_split = 0.2)
        BSNN_train_pred = model.predict(X_train)
        BSNN_test_pred = model.predict(X_test)
        trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
        testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
        score_small.append((trainr2, testr2))
    score.append(score_small)
score

[[(0.9431, 0.9356), (0.9248, 0.9216)],
 [(0.8955, 0.8917), (0.9446, 0.9387)],
 [(0.8326, 0.817), (0.9301, 0.9299)],
 [(0.9111, 0.9069), (0.9186, 0.9122)],
 [(0.9238, 0.9149), (0.9426, 0.9361)],
 [(0.899, 0.898), (0.9456, 0.9412)],
 [(0.9246, 0.92), (0.9355, 0.9326)],
 [(0.9099, 0.9054), (0.93, 0.9233)],
 [(0.7471, 0.749), (0.9631, 0.9602)],
 [(0.6701, 0.6807), (0.9526, 0.9508)]]

In [15]:
learning_rate = [1e-2, 1e-3]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 5,
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    
    numLayers = 3
    for i in range(numLayers):
        model.add(BatchNormalization())
        model.add(layers.Dense(units = 100, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))
        
    model.add(BatchNormalization())
    model.add(layers.Dense(units = 24, 
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    print("The training R squared is", 
          np.round(r2_score(y_train, BSNN_train_pred), 4))
    print("The testing R squared is", 
          np.round(r2_score(y_test, BSNN_test_pred), 4))
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.8802
The testing R squared is 0.8804
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.9434
The testing R squared is 0.9378
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_withLR0.001/assets
